<a href="https://colab.research.google.com/github/anms2024/DRL-Assignment-Group191/blob/create/Copy_of_FrozenLake_using_Dynamic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Group ID:191
### Group Members Name with Student ID:
1. KAUSHAL RAJKOTIA - 2023ad05029@wilp.bits-pilani.ac.in
2. M S ANJANA - 2023ac05498@wilp.bits-pilani.ac.in
3. NAGENDRA KUMAR  - 2023ac05904@wilp.bits-pilani.ac.in


1.**Problem statement**:

* Develop a reinforcement learning agent using dynamic programming to solve the Treasure Hunt problem in a FrozenLake environment. The agent must learn the optimal policy for navigating the lake while avoiding holes and maximizing its treasure collection.

2.**Scenario**:
* A treasure hunter is navigating a slippery 5x5 FrozenLake grid. The objective is to navigate through the lake collecting treasures while avoiding holes and ultimately reaching the exit (goal).
Grid positions on a 5x5 map with tiles labeled as S, F, H, G, T. The state includes the current position of the agent and whether treasures have been collected.


#### Objective
* The agent must learn the optimal policy π* using dynamic programming to maximize its cumulative reward while navigating the lake.

#### About the environment

The environment consists of several types of tiles:
* Start (S): The initial position of the agent, safe to step.
* Frozen Tiles (F): Frozen surface, safe to step.
* Hole (H): Falling into a hole ends the game immediately (die, end).
* Goal (G): Exit point; reaching here ends the game successfully (safe, end).
* Treasure Tiles (T): Added to the environment. Stepping on these tiles awards +5 reward but does not end the game.

After stepping on a treasure tile, it becomes a frozen tile (F).
The agent earns rewards as follows:
* Reaching the goal (G): +10 reward.
* Falling into a hole (H): -10 reward.
* Collecting a treasure (T): +5 reward.
* Stepping on a frozen tile (F): 0 reward.

#### States
* Current position of the agent (row, column).
* A boolean flag (or equivalent) for whether each treasure has been collected.

#### Actions
* Four possible moves: up, down, left, right

#### Rewards
* Goal (G): +10.
* Treasure (T): +5 per treasure.
* Hole (H): -10.
* Frozen tiles (F): 0.

#### Environment
Modify the FrozenLake environment in OpenAI Gym to include treasures (T) at certain positions. Inherit the original FrozenLakeEnv and modify the reset and step methods accordingly.
Example grid:

![image.png](attachment:image.png)


**Expected Outcomes:**
1.	Create the custom environment by modifying the existing “FrozenLakeNotSlippery-v0” in OpenAI Gym and Implement the dynamic programming using value iteration and policy improvement to learn the optimal policy for the Treasure Hunt problem.
2.	Calculate the state-value function (V*) for each state on the map after learning the optimal policy.
3.	Compare the agent’s performance with and without treasures, discussing the trade-offs in reward maximization.
4.	Visualize the agent’s direction on the map using the learned policy.
5.	Calculate expected total reward over multiple episodes to evaluate performance.

### Import required libraries and Define the custom environment - 2 Marks

In [ ]:
# Import statements
import gym
from gym import spaces
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Custom environment to create the given grid and respective functions that are required for the problem
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv

grid = [
    "SFFHT",
    "FHFFF",
    "FFFTF",
    "TFHFF",
    "FFFFG"
]
class TreasureHuntFrozenLake(FrozenLakeEnv):
    def __init__(self, desc=grid, map_name="5x5", is_slippery=False): # fix: Renamed _init_ to __init__
        super().__init__(desc=desc, map_name=map_name, is_slippery=is_slippery)
        self.treasures = set()  # Store treasure positions

    def reset(self):
        obs = super().reset()
        self.treasures = {(1, 2), (2, 1)}  # Example treasure positions
        return obs

    def step(self, action):
        # Fix: Modified to handle the case when the parent class returns 3 values
        # Assuming obs, reward, done,truncated, info are returned by the parent class's step method

        step_result = super().step(action) # The parent step method might be returning 5 values
        # Always unpack 5 values, as FrozenLakeEnv.step returns 5
        obs, reward, done, truncated, info = step_result


        row, col = divmod(self.s, self.ncol)

        # Check for treasure
        if (row, col) in self.treasures:
            reward += 5
            self.treasures.remove((row, col))  # Remove treasure
            self.desc[row, col] = b'F'  # Change to frozen tile
        return obs, reward, done, truncated, info # Always return 5 values


### Value Iteration Algorithm - 1 Mark

In [ ]:
def value_iteration(env, gamma=0.9, theta=1e-5):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    V = np.zeros(n_states)  # Initialize state-value function

    while True:
        delta = 0
        for s in range(n_states):
            v = V[s]
            q_values = []
            for a in range(n_actions):
                q_value = 0
                for prob, next_state, reward, done in env.P[s][a]:
                    q_value += prob * (reward + gamma * V[next_state] * (not done))
                q_values.append(q_value)
            V[s] = max(q_values)
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break

    # Derive the optimal policy
    policy = np.zeros([n_states, n_actions])
    for s in range(n_states):
        q_values = np.zeros(n_actions)
        for a in range(n_actions):
            for prob, next_state, reward, done in env.P[s][a]:
                q_values[a] += prob * (reward + gamma * V[next_state] * (not done))
        policy[s] = np.eye(n_actions)[np.argmax(q_values)]

    return V, policy

### Policy Improvement Function - 1 Mark

In [ ]:
def policy_improvement(env, policy, V, gamma=0.9):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    policy_stable = True

    for s in range(n_states):
        old_action = np.argmax(policy[s])
        q_values = np.zeros(n_actions)
        for a in range(n_actions):
            for prob, next_state, reward, done in env.P[s][a]:
                q_values[a] += prob * (reward + gamma * V[next_state] * (not done))
        new_action = np.argmax(q_values)
        if new_action != old_action:
            policy_stable = False
        policy[s] = np.eye(n_actions)[new_action]
    return policy, policy_stable

### Print the Optimal Value Function

In [ ]:
# Create the environment
env = TreasureHuntFrozenLake()

# Run value iteration
V, policy = value_iteration(env)

# Print the optimal value function
print("Optimal Value Function:")
print(V.reshape(env.nrow, env.ncol))

Optimal Value Function:
[[0.4782969 0.531441  0.59049   0.        0.729    ]
 [0.531441  0.        0.6561    0.729     0.81     ]
 [0.59049   0.6561    0.729     0.81      0.9      ]
 [0.6561    0.729     0.        0.9       1.       ]
 [0.729     0.81      0.9       1.        0.       ]]


### Visualization of the learned optimal policy - 1 Mark

In [ ]:
# Policy visualization
def visualize_policy(policy, env):
    directions = ['←', '↓', '→', '↑']
    grid = np.array(env.desc, dtype='str')
    for s, actions in enumerate(policy):
        row, col = divmod(s, env.ncol)
        grid[row, col] = directions[np.argmax(actions)]
    return "\n".join("".join(row) for row in grid)

### Evaluate the policy - 1 Mark

In [ ]:
# Policy evaluation
def evaluate_policy(env, policy, episodes=1000):
    total_rewards = []
    for _ in range(episodes):
        obs = env.reset()
        done = False
        rewards = 0
        while not done:
            action = np.argmax(policy[obs])
            # Modified to unpack 5 values returned by env.step
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated # Set done if either terminated or truncated is True
            rewards += reward
        total_rewards.append(rewards)
    return np.mean(total_rewards)

### Main Execution

In [ ]:
# Create environments
env_with_treasures = TreasureHuntFrozenLake(map_name="4x4") # Change map_name to "4x4" or "8x8"
# Use '4x4' or '8x8' instead of '5x5'
env_without_treasures = FrozenLakeEnv(map_name="4x4", is_slippery=False)

# Value iteration
_, policy_with_treasures = value_iteration(env_with_treasures)
_, policy_without_treasures = value_iteration(env_without_treasures)

# Evaluate policies
reward_with_treasures = evaluate_policy(env_with_treasures, policy_with_treasures)
reward_without_treasures = evaluate_policy(env_without_treasures, policy_without_treasures)

# Visualize policies
policy_visual_with_treasures = visualize_policy(policy_with_treasures, env_with_treasures)
policy_visual_without_treasures = visualize_policy(policy_without_treasures, env_without_treasures)

# Output results
print("Policy with treasures:\n", policy_visual_with_treasures)
print("\nPolicy without treasures:\n", policy_visual_without_treasures)
print("\nExpected total reward with treasures:", reward_with_treasures)
print("Expected total reward without treasures:", reward_without_treasures)

Policy with treasures:
 ↓→↓←↓
↓←↓↓↓
↓↓→↓↓
↓↓←↓↓
→→→→←

Policy without treasures:
 ↓→↓←
↓←↓←
→↓↓←
←→→←

Expected total reward with treasures: 1.0
Expected total reward without treasures: 1.0
